In [1]:
import tensorflow as tf

In [3]:
# create a tensor
matrix = tf.constant([[1,2,3],[4,5,6]])
matrix

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [4]:
print(matrix.numpy())
print(matrix.shape)

[[1 2 3]
 [4 5 6]]
(2, 3)


In [5]:
# with a constont
matrix2 = matrix + 2
matrix2

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[3, 4, 5],
       [6, 7, 8]], dtype=int32)>

In [6]:
# with another tensor with the same shape
matrix + matrix2

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 4,  6,  8],
       [10, 12, 14]], dtype=int32)>

In [7]:
# relu
def my_relu(x):
  return tf.maximum(0, x)

# sigmoid
def my_sigmoid(x):
  return 1/(1+tf.exp(-x))

# softmax
def my_softmax(x):
  expo = tf.exp(x)
  expo_sum = tf.sum(tf.exp(x))
  return expo/expo_sum

# softplus
def my_softplus(x):
  return tf.math.log(tf.exp(x) + 1.0)

In [8]:
# mean squared error
def mse(y_true, y_pred):
    error = y_true - y_pred
    squared_error = tf.square(error)
    return tf.mean(squared_error)

In [9]:
# OOP approach required if function has a hyper parameter
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [10]:
# Functional Programming approach
def my_l1_regularizer(weight, factor=0.01):
    return tf.reduce_sum(tf.abs(factor * weight))

def my_l2_regularizer(weight, factor=0.01):
    return tf.reduce_sum(tf.square(factor * weight))

In [11]:
# OOP approach
class MyL1Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, weight):
        return tf.reduce_sum(tf.abs(self.factor * weight))

    def get_config(self):
        return {"factor": self.factor}

In [12]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal"
        )
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros"
        )
        super().build(batch_input_shape)

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": tf.keras.activations.serialize(self.activation)}